In [1]:
%reset -f

In [2]:
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn import svm, datasets, feature_selection, cross_validation
from sklearn.feature_selection import SelectKBest, f_regression, SelectFdr, SelectFpr
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, RFECV, RFE, mutual_info_classif
from sklearn.feature_selection import VarianceThreshold, f_regression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import validation_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib
from sklearn.metrics import f1_score, precision_score,recall_score, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn import feature_selection
from sklearn import cross_validation
from sklearn.externals import joblib
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,OneHotEncoder,Imputer,Normalizer
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.externals import joblib
from sklearn.decomposition import PCA, RandomizedPCA, KernelPCA
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.learning_curve import learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.externals import joblib
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LassoCV
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC

from scipy.stats import mannwhitneyu
from scipy.stats import pearsonr, spearmanr

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from pylab import pcolor, show, colorbar, xticks, yticks
from numpy import corrcoef, sum, log, arange

from multiprocessing import Pool

import matplotlib.pyplot as plt
import logging.config
import seaborn as sns
import pandas as pd
import numpy as np 
import datetime
import time
import csv
import os
import pickle
%matplotlib inline

import sys
sys.path.insert(1, "/home/aegle/projects/myosa/src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score
from UnivCombineFilter import UnivCombineFilter

/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. Th

In [3]:
folder = "equal_more_four" #"more_than_four"

In [25]:
ds = 2

In [26]:
datafilenames = []
datafilenames.append(os.path.join('resources','data_partition_cl0_short_no_monit_14012016.pkl'))
datafilenames.append(os.path.join('resources','data_partition_cl0_short_1m_monit_14012016.pkl'))
datafilenames.append(os.path.join('resources','data_partition_cl0_short_3m_1m_monit_14012016.pkl'))


#Load Train & test data
f = file(datafilenames[ds],"rb")
X_train = np.load(f).astype(float)
y_train = np.load(f).astype(int)
X_test = np.load(f).astype(float)
y_test = np.load(f).astype(int)
cols = np.load(f)
tr_codes = np.load(f)
ts_codes = np.load(f)
feat_types = np.load(f)
feat_types = dict(feat_types.tolist())
f.close()

#Join data
X_all = np.vstack((X_train,X_test))
y_all = np.hstack((y_train, y_test)).reshape(-1,1)
all_codes = np.hstack((tr_codes,ts_codes))
print X_all.shape, y_all.shape, all_codes.shape
print X_train.shape, X_test.shape
print cols

print "RONCA:", np.unique(X_all[:,cols=="RONCA"])
print "CEFALEA:", np.unique(X_all[:,cols=="CEFALEA"])

(42, 70) (42, 1) (42,)
(29, 70) (13, 70)
['GENDER' 'ACTIVE' 'RETIRED' 'AGE' 'SMOKER' 'SMOKE_PACK_YEAR'
 'SMOKE_EX_YEAR' 'ALCOHOL' 'ALCOHOL_GR_DAY' 'CAFES_DAY' 'DEPRESSION'
 'ANXIETY' 'HTA' 'CARDIOPATHY' 'RESP_DIS' 'DIABETES' 'OBESITY' 'DISLIPEMIA'
 'OBSTR_NASAL_CHRON' 'SNORES' 'REP_SLEEP' 'CRISIS_ASP' 'NOCTURIA'
 'APNEAS_PRES' 'CEFALEA' 'DAY_SOMN' 'AWAKE_NIGHT' 'CONC_DIS'
 'IRR_APAT_DEPRE' 'INSOMNIA' 'ACT_MOTR_NOCT' 'FEEL_REAL_DRE'
 'DREAM_INT_PIROSIS' 'DEC_SEX_DIS' 'IECAS' 'DIURETICS' 'ANTIAGREG'
 'ANTIACID' 'HIPOLIPEM' 'BDZ' 'SLEEP_TIME' 'IAH' 'TIME_SAT<90%' 'IND_DESAT'
 'SIZE' 'BMI' 'CIRC_NECK' 'TAS_MEAN' 'TAD_MEAN' 'SAT_O2' 'PRESS_CPAP'
 'EPWORTH' 'EUROQOL' 'VISUAL' 'TOTAL_USE_HOURS_1' 'MEAN_USE_HOURS_1'
 'AL_IRR_CUT_MASC_1' 'DRY_MOUTH_1' 'TAS_MEAN_1' 'TAD_MEAN_1' 'EPWORTH_1'
 'EUROQOL_1' 'VISUAL_EUROQOL_1' 'MEAN_USE_HOURS_3' 'DRY_MOUTH_3'
 'TAS_MEAN_3' 'TAD_MEAN_3' 'EPWORTH_3' 'EUROQOL_3' 'VISUAL_EUROQOL_3'
 'LABEL']
RONCA: []
CEFALEA: [ 0.  1.]


/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/ipykernel/__main__.py:28: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 70 but corresponding boolean dimension is 71
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/ipykernel/__main__.py:29: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 70 but corresponding boolean dimension is 71


#### Find categorical (discrete) columns

In [27]:
catCols = []
reducedCols = cols[:-1]

for i in range(len(cols)-1):
    if feat_types[cols[i]] == 'object' \
        or cols[i] in ("RONCA","SUE_REP","CRISIS_ASF","NICTURIA","APNEAS_PRES","CEFALEA",
                "SOMN_DIURN","DESPERTAR_NOCT","TRAST_MEM","TRAST_CONC","IRR_APAT_DEPR"):
        catCols.append(1)
    else:
        catCols.append(0)
catCols = np.array(catCols)

print "Cat cols:", np.sum(catCols==1)
print "Num cols:", np.sum(catCols==0)
print len(reducedCols)

Cat cols: 30
Num cols: 40
70


#### Config pipelines

In [28]:
verbose = False
fs_methods = ["none","combine_fs","lasso_fs","rfe_rf_fs"]
cls_methods = ["rf","svmRBF","logReg","knn","nn"]
metrics = ["f1_weighted","precision_weighted"]
sm_types = ["none","after"]
suffixes = ['none'] #,'outlier'
sm_method = "sm_smote"

#### Create pipelines

In [29]:
basePipeline = Pipeline([
        ("Imputer", TypeFeatImputer(catCols, reducedCols)),
        ("Scaler", StandardScaler()),
        ("Variance", VarianceThreshold(threshold=0.0))
    ])

In [30]:
pipeline = [] 
for suffix in suffixes:
    for fs_method in fs_methods:
        for sm_type in sm_types:
            for cls_method in cls_methods:
                for metric in metrics:
                    
                    if not (fs_method == "rfe_rf_fs" and cls_method == "rf"):
                        params = {}   
                        pipe = Pipeline(list(basePipeline.steps))

                        if fs_method == "combine_fs":
                            pipe.steps.insert(1,(fs_method, UnivCombineFilter(catCols,np.array(reducedCols))))
                            params.update({fs_method + '__percentile':[5,10,20,30,40,50]})
        
                        if fs_method == "rfe_rf_fs":
                            pipe.steps.append((fs_method, RFE(estimator=RandomForestClassifier(class_weight='balanced',
                                                                                               n_estimators=100,
                                                                                               random_state=33))))
                            params.update({fs_method + '__step':[0.1]})
                            params.update({fs_method + '__n_features_to_select':[
                                                            int(len(reducedCols)*0.4), 
                                                            int(len(reducedCols)*0.6), 
                                                            int(len(reducedCols)*0.8)]})

                        if fs_method == 'lasso_fs':
                            pipe.steps.append((fs_method, SelectFromModel(
                                        LogisticRegression(n_jobs=-1, penalty="l1", dual=False, random_state=42))))
                            params.update({fs_method + '__estimator__C': [0.001,0.01,0.1,1]})

                        #Add classifiers
                        if cls_method == "knn":
                            pipe.steps.append((cls_method, KNeighborsClassifier()))
                            params.update({'knn__n_neighbors':[1,3,5,7,9,11], 'knn__weights':['uniform', 'distance']})

                        if cls_method == "logReg":
                            pipe.steps.append((cls_method, LogisticRegression(random_state=42)))
                            params.update({'logReg__C': [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,15,30]})
                            params.update({'logReg__class_weight': [None, 'balanced']})
                            params.update({'logReg__penalty': ['l1', 'l2']})

                        if cls_method == "svmRBF":
                            pipe.steps.append((cls_method, SVC(random_state=42,probability=True)))
                            params.update({'svmRBF__C': [0.01,0.1,0.5,1,5,10,15,30,50], 
                             'svmRBF__gamma' : [0.0001,0.001,0.01, 0.1,1,5]})
                            params.update({'svmRBF__class_weight': [None, 'balanced']})

                        if cls_method == "rf":
                            pipe.steps.append((cls_method, RandomForestClassifier(n_jobs=-1,random_state=42)))
                            params.update({'rf__n_estimators': [100,150,200,250,500], 
                                           'rf__criterion': ['entropy','gini'],
                                           'rf__max_depth' : [None,4,6]})
                            params.update({'rf__class_weight': [None, 'balanced']})

                        if cls_method == "nn":
                            pipe.steps.append((cls_method, MLPClassifier(
                                        activation='logistic',
                                        solver='lbfgs', 
                                        hidden_layer_sizes=(5, 2), 
                                        random_state=13)))
                            params.update({
                                    'nn__alpha': [1e-5,0.00001,0.0001,0.001,0.01,0.1,1,3,5,10],
                                    'nn__hidden_layer_sizes':[(30,),(50,),(70,),(100,),(150,),
                                                              (30,30),(50,50),(70,70),(100,100),
                                                              (30,30,30),(50,50,50),(70,70,70)
                                                             ]
                                          })
                            
                        #Add sampling
                        pipe_imb = make_pipeline(*[p[1] for p in pipe.steps])
                        stps = len(pipe_imb.steps)        
                        for s in range(stps):
                            pipe_imb.steps.remove(pipe_imb.steps[0])
                        for s in range(stps):
                            pipe_imb.steps.append(pipe.steps[s])

                        if sm_type == "after":                    
                            pipe_imb.steps.insert(stps - 1, 
                                                  (sm_method, SMOTE(ratio='auto', kind='regular', random_state=32)))
                            params.update({sm_method + "__k_neighbors":[3,4,5]})


                        pipeline.append([suffix,fs_method,sm_type,cls_method,metric,pipe_imb,params])

In [31]:
pipeline = pd.DataFrame(pipeline, columns=["out","fs","sm","cls","metric","pipe","pipe_params"])
pipeline.sort_values("fs", inplace=True)
print pipeline.shape
pipeline

(76, 7)


,out,fs,sm,cls,metric,pipe,pipe_params
37,none,combine_fs,after,knn,precision_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'knn__weights': [u'uniform', u'distance'], u..."
21,none,combine_fs,none,rf,precision_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'rf__criterion': [u'entropy', u'gini'], u'rf..."
22,none,combine_fs,none,svmRBF,f1_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'svmRBF__gamma': [0.0001, 0.001, 0.01, 0.1, ..."
23,none,combine_fs,none,svmRBF,precision_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'svmRBF__gamma': [0.0001, 0.001, 0.01, 0.1, ..."
24,none,combine_fs,none,logReg,f1_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'logReg__class_weight': [None, u'balanced'],..."
25,none,combine_fs,none,logReg,precision_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'logReg__class_weight': [None, u'balanced'],..."
26,none,combine_fs,none,knn,f1_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'knn__weights': [u'uniform', u'distance'], u..."
27,none,combine_fs,none,knn,precision_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'knn__weights': [u'uniform', u'distance'], u..."
20,none,combine_fs,none,rf,f1_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'rf__criterion': [u'entropy', u'gini'], u'rf..."
28,none,combine_fs,none,nn,f1_weighted,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'nn__hidden_layer_sizes': [(30,), (50,), (70..."


#### Run all experiments

In [32]:
print "ALL TRAIN:", X_train.shape
print "TRAIN:", "[0's:", np.sum(y_train==0), "1's:", np.sum(y_train==1), "]"
print "ALL TEST:", X_test.shape
print "TEST:", "[0's:", np.sum(y_test==0), "1's:", np.sum(y_test==1), "]"

results = []
for num_exp in range(pipeline.shape[0]):

    # Run experiment
    start = time.time()

    #Prepare pipe_cls      
    pipeline_cls = pipeline["pipe"].iloc[num_exp]
    pipeline_params = pipeline["pipe_params"].iloc[num_exp]
    out = pipeline["out"].iloc[num_exp]
    fs = pipeline["fs"].iloc[num_exp]
    sm = pipeline["sm"].iloc[num_exp]
    cls = pipeline["cls"].iloc[num_exp]
    metric = pipeline["metric"].iloc[num_exp]

    print "\n Num experiment:", num_exp
    print "****************"
    print out
    print fs
    print sm
    print cls
    print metric


    if out == "outlier":
        y_train_aux = y_train_out
        y_test_aux = y_test_out
        X_train_aux = X_train_out
        X_test_aux = X_test_out
    else:
        y_train_aux = y_train
        y_test_aux = y_test
        X_train_aux = X_train
        X_test_aux = X_test

    #Prepare cv
    cv_inner = StratifiedShuffleSplit(y_train_aux, n_iter=10, test_size=0.30, random_state=24)
    cv_outer = StratifiedShuffleSplit(y_train_aux, n_iter=10, test_size=0.30, random_state=42)


    #Fit pipeline with CV                        
    grid_pipeline = GridSearchCV(pipeline_cls, param_grid=pipeline_params, verbose=0, 
                                 n_jobs=-1, cv=cv_inner, scoring= metric, error_score = 0) 
    grid_pipeline.fit(X_train_aux, y_train_aux)

    # Compute pipeline evaluation with CV
    print "\nCV INNER metric: {}".format(metric)
    print "CV INNER selected params {}".format(grid_pipeline.best_params_.values())
    print "CV INNER score: {}".format(grid_pipeline.best_score_)

    cv_f1 = cross_validation.cross_val_score(grid_pipeline.best_estimator_, X_train_aux, y_train_aux, 
                                             cv=cv_outer, scoring='f1_weighted', n_jobs=-1)

    cv_prec = cross_validation.cross_val_score(grid_pipeline.best_estimator_, X_train_aux, y_train_aux, 
                                             cv=cv_outer, scoring='precision_weighted', n_jobs=-1)

    cv_rec = cross_validation.cross_val_score(grid_pipeline.best_estimator_, X_train_aux, y_train_aux, 
                                             cv=cv_outer, scoring='recall_weighted', n_jobs=-1)
    
    print "\nCV OUTER f1 score: %0.3f  (+/-%0.03f)" % (np.mean(cv_f1), np.std(cv_f1))
    print "CV OUTER prec score: %0.3f  (+/-%0.03f)" % (np.mean(cv_prec), np.std(cv_prec))
    print "CV OUTER rec score: %0.3f  (+/-%0.03f)" % (np.mean(cv_rec), np.std(cv_rec))
    print "Selected params (bests from CV) {}".format(grid_pipeline.best_params_.values())

    # Computel Train score (with best CV params)
    y_pred = grid_pipeline.best_estimator_.predict(X_train_aux)
    train_prec_scores = precision_score(y_train_aux, y_pred, average='weighted', pos_label=None)
    train_rec_scores = recall_score(y_train_aux, y_pred, average='weighted', pos_label=None)    
    train_f1_scores = f1_score(y_train_aux, y_pred, average='weighted', pos_label=None)

    print "\nTR F1 score:", train_f1_scores
    print "TR Prec score:", train_prec_scores
    print "TR Rec score:", train_rec_scores

    #Compute test score
    y_pred = grid_pipeline.best_estimator_.predict(X_test_aux)
    test_f1 = f1_score(y_test_aux, y_pred, average='weighted', pos_label=None)
    test_prec = recall_score(y_test_aux, y_pred, average='weighted', pos_label=None)
    test_rec = precision_score(y_test_aux, y_pred, average='weighted', pos_label=None)
    test_auc = roc_auc_score(y_test_aux, y_pred, average='weighted')
    
    print "\nTest f1: %0.3f" % (test_f1)
    print "Test Precision: %0.3f" % (test_prec)
    print "Test Recall: %0.3f" % (test_rec)
    print "Test AUC: %0.3f" % (test_auc)
    
    print "with following performance in test:"
    print classification_report(y_test_aux, y_pred)
    print confusion_matrix(y_test_aux, y_pred)

    end = time.time()
    print "\nTotal time:", end - start
    results.append([num_exp,
                   out,
                   fs,
                   sm,
                   cls,
                   metric,
                   grid_pipeline.best_params_.values(),
                   train_f1_scores,
                   train_prec_scores,
                   train_rec_scores,
                   np.mean(cv_f1), 
                   np.std(cv_f1),
                   np.mean(cv_prec), 
                   np.std(cv_prec),
                   np.mean(cv_rec), 
                   np.std(cv_rec),                    
                   test_f1,
                   test_prec,
                   test_rec,
                   test_auc,                    
                   end - start,
                   grid_pipeline.best_estimator_
                  ])

    #Save results
    df = pd.DataFrame(results, columns=
              ["exp","out","fs","sm","cls","metric","params",
               "tr_f1","tr_prec","tr_rec",
               "cv_f1_mean","cv_f1_std","cv_prec_mean","cv_prec_std","cv_rec_mean","cv_rec_std",
               "test_f1","test_prec","test_rec","test_auc",
               "time","pipeline"])
    df.to_pickle(os.path.join('resources','results',folder,'ds_'+str(ds),'results_pipe_' + str(ds) + '_' + str(num_exp) + '.pkl'))
    

ALL TRAIN: (29, 70)
TRAIN: [0's: 12 1's: 17 ]
ALL TEST: (13, 70)
TEST: [0's: 6 1's: 7 ]

 Num experiment: 0
****************
none
combine_fs
after
knn
precision_weighted


/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predict


CV INNER metric: precision_weighted
CV INNER selected params ['uniform', 11, 5, 4]
CV INNER score: 0.792843915344

CV OUTER f1 score: 0.741  (+/-0.100)
CV OUTER prec score: 0.758  (+/-0.107)
CV OUTER rec score: 0.744  (+/-0.100)
Selected params (bests from CV) ['uniform', 11, 5, 4]

TR F1 score: 0.931034482759
TR Prec score: 0.931034482759
TR Rec score: 0.931034482759

Test f1: 0.922
Test Precision: 0.923
Test Recall: 0.933
Test AUC: 0.917
with following performance in test:
             precision    recall  f1-score   support

          0       1.00      0.83      0.91         6
          1       0.88      1.00      0.93         7

avg / total       0.93      0.92      0.92        13

[[5 1]
 [0 7]]

Total time: 22.7675161362

 Num experiment: 1
****************
none
combine_fs
none
rf
precision_weighted


/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predict


CV INNER metric: precision_weighted
CV INNER selected params [200, 'entropy', None, None, 20]
CV INNER score: 0.838492063492

CV OUTER f1 score: 0.838  (+/-0.118)
CV OUTER prec score: 0.865  (+/-0.111)
CV OUTER rec score: 0.844  (+/-0.113)
Selected params (bests from CV) [200, 'entropy', None, None, 20]

TR F1 score: 1.0
TR Prec score: 1.0
TR Rec score: 1.0

Test f1: 0.751
Test Precision: 0.769
Test Recall: 0.838
Test AUC: 0.750
with following performance in test:
             precision    recall  f1-score   support

          0       1.00      0.50      0.67         6
          1       0.70      1.00      0.82         7

avg / total       0.84      0.77      0.75        13

[[3 3]
 [0 7]]

Total time: 423.849259138

 Num experiment: 2
****************
none
combine_fs
none
svmRBF
f1_weighted


...


CV INNER metric: f1_weighted
CV INNER selected params [28, (70, 70, 70), 0.1, 4, 1e-05]
CV INNER score: 0.693431938432

CV OUTER f1 score: 0.666  (+/-0.175)
CV OUTER prec score: 0.717  (+/-0.181)
CV OUTER rec score: 0.678  (+/-0.168)
Selected params (bests from CV) [28, (70, 70, 70), 0.1, 4, 1e-05]

TR F1 score: 1.0
TR Prec score: 1.0
TR Rec score: 1.0

Test f1: 0.751
Test Precision: 0.769
Test Recall: 0.838
Test AUC: 0.750
with following performance in test:
             precision    recall  f1-score   support

          0       1.00      0.50      0.67         6
          1       0.70      1.00      0.82         7

avg / total       0.84      0.77      0.75        13

[[3 3]
 [0 7]]

Total time: 1787.54068995

 Num experiment: 74
****************
none
rfe_rf_fs
none
nn
f1_weighted


/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', ave


CV INNER metric: f1_weighted
CV INNER selected params [56, (50,), 0.1, 0.001]
CV INNER score: 0.67418951419

CV OUTER f1 score: 0.760  (+/-0.119)
CV OUTER prec score: 0.795  (+/-0.120)
CV OUTER rec score: 0.767  (+/-0.116)
Selected params (bests from CV) [56, (50,), 0.1, 0.001]

TR F1 score: 1.0
TR Prec score: 1.0
TR Rec score: 1.0

Test f1: 0.844
Test Precision: 0.846
Test Recall: 0.885
Test AUC: 0.857
with following performance in test:
             precision    recall  f1-score   support

          0       0.75      1.00      0.86         6
          1       1.00      0.71      0.83         7

avg / total       0.88      0.85      0.84        13

[[6 0]
 [2 5]]

Total time: 605.566128969

 Num experiment: 75
****************
none
rfe_rf_fs
after
nn
precision_weighted


/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predict


CV INNER metric: precision_weighted
CV INNER selected params [28, (70, 70, 70), 0.1, 4, 1e-05]
CV INNER score: 0.727142857143

CV OUTER f1 score: 0.666  (+/-0.175)
CV OUTER prec score: 0.717  (+/-0.181)
CV OUTER rec score: 0.678  (+/-0.168)
Selected params (bests from CV) [28, (70, 70, 70), 0.1, 4, 1e-05]

TR F1 score: 1.0
TR Prec score: 1.0
TR Rec score: 1.0

Test f1: 0.751
Test Precision: 0.769
Test Recall: 0.838
Test AUC: 0.750
with following performance in test:
             precision    recall  f1-score   support

          0       1.00      0.50      0.67         6
          1       0.70      1.00      0.82         7

avg / total       0.84      0.77      0.75        13

[[3 3]
 [0 7]]

Total time: 1755.13114786


In [ ]:
df = pd.DataFrame(results, columns=
                  ["exp","out","fs","sm","cls","metric","params",
               "tr_f1","tr_prec","tr_rec",
               "cv_f1_mean","cv_f1_std","cv_prec_mean","cv_prec_std","cv_rec_mean","cv_rec_std",
               "test_f1","test_prec","test_rec","test_auc",
               "time","pipeline"])

In [15]:
df.sort_values("cv_f1_mean", ascending=False).head(10)

,exp,out,fs,sm,cls,metric,params,tr_f1,tr_prec,tr_rec,...,cv_prec_mean,cv_prec_std,cv_rec_mean,cv_rec_std,test_f1,test_prec,test_rec,test_auc,time,pipeline
50,50,none,none,none,svmRBF,f1_weighted,"[0.001, None, 50]",1.000000,1.000000,1.000000,...,0.760847,0.202788,0.722222,0.166667,0.760839,0.769231,0.846154,0.785714,8.322935,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
48,48,none,none,none,nn,f1_weighted,"[(30, 30), 0.01]",1.000000,1.000000,1.000000,...,0.760847,0.202788,0.722222,0.166667,0.760839,0.769231,0.846154,0.785714,19.046475,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
43,43,none,none,none,svmRBF,precision_weighted,"[0.001, None, 50]",1.000000,1.000000,1.000000,...,0.760847,0.202788,0.722222,0.166667,0.760839,0.769231,0.846154,0.785714,8.974537,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
44,44,none,none,none,logReg,f1_weighted,"[None, 30, l2]",1.000000,1.000000,1.000000,...,0.750635,0.173921,0.711111,0.158698,0.760839,0.769231,0.846154,0.785714,5.270101,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
45,45,none,none,none,logReg,precision_weighted,"[None, 30, l2]",1.000000,1.000000,1.000000,...,0.750635,0.173921,0.711111,0.158698,0.760839,0.769231,0.846154,0.785714,5.629596,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
53,53,none,none,after,svmRBF,precision_weighted,"[0.001, None, 3, 50]",1.000000,1.000000,1.000000,...,0.751958,0.193862,0.711111,0.150718,0.760839,0.769231,0.846154,0.785714,26.899843,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
52,52,none,none,after,svmRBF,f1_weighted,"[0.001, None, 3, 50]",1.000000,1.000000,1.000000,...,0.751958,0.193862,0.711111,0.150718,0.760839,0.769231,0.846154,0.785714,27.291420,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
58,58,none,none,after,nn,f1_weighted,"[(50, 50), 4, 0.01]",1.000000,1.000000,1.000000,...,0.767566,0.145940,0.711111,0.133333,0.760839,0.769231,0.846154,0.785714,54.268251,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
49,49,none,none,none,nn,precision_weighted,"[(150,), 0.01]",1.000000,1.000000,1.000000,...,0.756614,0.201281,0.711111,0.166296,0.760839,0.769231,0.846154,0.785714,19.525650,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."
12,12,none,combine_fs,after,svmRBF,f1_weighted,"[0.001, None, 3, 40, 15]",0.931034,0.931034,0.931034,...,0.698042,0.203361,0.688889,0.184592,0.565108,0.615385,0.790210,0.642857,178.730455,"Pipeline(steps=[('Imputer', TypeFeatImputer(al..."


In [ ]:
df.to_pickle(os.path.join('resources','results','ds-{}.pkl'.format(ds)))